#First attempt
First attempt failed, because tiny-start_coder was trained on python. Skip to [JetBrains AI Code Completion Solution section](https://colab.research.google.com/drive/11H22yOkHZWbX20RlC7AiA5JYGbQUvpa_#scrollTo=CvoMLWuO3Y7Z&line=1&uniqifier=1).

In [ ]:
!git clone https://github.com/Kasprian/stayfit.git

Cloning into 'stayfit'...
remote: Enumerating objects: 158, done.
remote: Total 158 (delta 0), reused 0 (delta 0), pack-reused 158 (from 1)
Receiving objects: 100% (158/158), 962.73 KiB | 23.48 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [ ]:
!pip install python-Levenshtein nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.7 MB/s eta 0:00:00


In [ ]:
ls /content/

sample_data/  stayfit/


In [ ]:
import os
import random

def generate_completion_examples(file_paths, num_examples=50, min_prefix=10, min_suffix=10):
    examples = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for _ in range(num_examples):
                line_num = random.randint(0, len(lines) - 1)
                line = lines[line_num]
                # Skip lines shorter than the combined minimum length of prefix and suffix
                if len(line) < min_prefix + min_suffix:
                    continue
                cursor_pos = random.randint(min_prefix, len(line) - min_suffix)
                prefix = line[:cursor_pos]
                middle = line[cursor_pos:cursor_pos+min_suffix]  # assume small length as 'middle'
                suffix = line[cursor_pos+min_suffix:]
                examples.append((prefix, middle, suffix))
    return examples

# Usage:
files = [
    '/content/stayfit/src/main/java/App.java',
    '/content/stayfit/src/main/java/Controllers/AddExerciseController.java',
    '/content/stayfit/src/main/java/Controllers/BMIStateController.java',
    '/content/stayfit/src/main/java/Controllers/DietPlanController.java',
    '/content/stayfit/src/main/java/Controllers/LoginScreenController.java',
    '/content/stayfit/src/main/java/Model/BMI_state.java',
    '/content/stayfit/src/main/java/Model/User_size.java',
    '/content/stayfit/src/main/java/Model/Training_plan.java',

]
dataset = generate_completion_examples(files)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigcode/tiny_starcoder_py"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_code_completion(model, tokenizer, prefix, max_new_tokens=20):
    inputs = tokenizer(prefix, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Generate completions
completions = []
for prefix, middle, suffix in dataset:
    generated = generate_code_completion(model, tokenizer, prefix)
    completions.append((prefix, middle, generated, suffix))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
labeled_completions = []
for prefix, middle, generated, suffix in completions:
    print(f"Prefix:\n{prefix}")
    print(f"Expected (Middle):\n{middle}")
    print(f"Generated:\n{generated}")
    label = input("Label (correct/partial/incorrect): ")
    labeled_completions.append((prefix, middle, generated, suffix, label))


Prefix:
        Scene scene = new S
Expected (Middle):
cene(root)
Generated:
        Scene scene = new SpheresScene(scene_id, scene_name, scene_description, scene_type,
Label (correct/partial/incorrect): incorrect
Prefix:
import javafx.s
Expected (Middle):
cene.Paren
Generated:
import javafx.sma.sma.sma.sma.sma.sma.sma.
Label (correct/partial/incorrect): incorrect
Prefix:
        stage.setResiza
Expected (Middle):
ble(false)
Generated:
        stage.setResiza(1)
        stage.setResiz(1)
        stage.setResiz(1
Label (correct/partial/incorrect): incorrect
Prefix:
        st
Expected (Middle):
age.setTit
Generated:
        st.write(f"**{name}**\n")
        st.write(f"**
Label (correct/partial/incorrect): incorrect
Prefix:
    public void start(
Expected (Middle):
Stage stag
Generated:
    public void start(void) {
    }

    public void stop(void) {
    }

    public void get_
Label (correct/partial/incorrect): partial
Prefix:
import jav
Expected (Middle):
afx.stage.
Generated:
import 

KeyboardInterrupt: Interrupted by user

In [ ]:
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.chrf_score import sentence_chrf
import Levenshtein

# Compute Metrics
exact_matches = [1 if gen == actual else 0 for _, actual, gen, _ in completions]
bleu_scores = [sentence_bleu([actual], gen) for _, actual, gen, _ in completions]
chrf_scores = [sentence_chrf(actual, gen) for _, actual, gen, _ in completions]
levenshtein_distances = [Levenshtein.distance(actual, gen) for _, actual, gen, _ in completions]

# Print results
print("Exact Match:", sum(exact_matches) / len(exact_matches))
print("Average BLEU Score:", sum(bleu_scores) / len(bleu_scores))
print("Average chrF Score:", sum(chrf_scores) / len(chrf_scores))
print("Average Levenshtein Distance:", sum(levenshtein_distances) / len(levenshtein_distances))


Exact Match: 0.0
Average BLEU Score: 0.01133151941187256
Average chrF Score: 0.12161680663860645
Average Levenshtein Distance: 82.97356828193833


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

##Results
tiny-start_coder was trained on Python files, and isn't suitable for Java. I will try Python project now.

#JetBrains AI Code Completion Solution
Now my random laboratory python project is taken into account.

In [ ]:
!git clone https://github.com/Kasprian/Applied-Computational-Intelligence.git

Cloning into 'Applied-Computational-Intelligence'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 77 (from 1)
Receiving objects: 100% (80/80), 2.42 MiB | 15.76 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [47]:
import os
import random

def generate_completion_examples(file_paths, total_examples=25, middle_length=10, min_prefix_tokens=3, min_suffix_tokens=0):
    examples = []
    for file_path in file_paths:
        if os.path.exists(file_path) and file_path.endswith('.py'):  # Ensure the file exists and is a .py file
            with open(file_path, 'r') as file:
                lines = file.readlines()
                while len(examples) < total_examples:  # Continue until we reach the desired number of examples
                    line_num = random.randint(0, len(lines) - 1)
                    line = lines[line_num]

                    # Skip lines shorter than the specified middle length
                    if len(line) < middle_length:
                        continue

                    # Choose a position for the middle section
                    cursor_pos = random.randint(0, len(line) - middle_length)
                    prefix = line[:cursor_pos].strip()  # All before the middle, trimmed to remove whitespace
                    middle = line[cursor_pos:cursor_pos + middle_length]  # Middle is exactly 10 characters
                    suffix = line[cursor_pos + middle_length:].strip()  # Only the rest of the line after middle

                    # Ensure the prefix and suffix have the minimum number of tokens
                    if len(prefix.split()) >= min_prefix_tokens and len(suffix.split()) >= min_suffix_tokens:
                        examples.append((prefix, middle, suffix))
    return examples

# Usage
files_aci = [
    '/content/Applied-Computational-Intelligence/project.py',
    '/content/Applied-Computational-Intelligence/tsp.py',
    '/content/Applied-Computational-Intelligence/CInt/project.py',
    '/content/Applied-Computational-Intelligence/Lab3/lab3_1.py',
    '/content/Applied-Computational-Intelligence/Lab3/lab3_2.py',
    '/content/Applied-Computational-Intelligence/Lab3/lab3_3.py',
    '/content/Applied-Computational-Intelligence/Lab4/lab4.py',
    '/content/Applied-Computational-Intelligence/Lab10/basic.py',
]

# Generate exactly 25 examples in total
dataset_aci = generate_completion_examples(files_aci, total_examples=25)

# Sample output to verify
for prefix, middle, suffix in dataset_aci[:5]:  # Display the first 5 examples
    print("Prefix:", prefix)
    print("Middle (Expected 10 chars):", middle)
    print("Suffix (At least 5 tokens):", suffix)
    print("---")


Prefix: Created on Wed Oct 20
Middle (Expected 10 chars): 22:57:12 2
Suffix (At least 5 tokens): 021
---
Prefix: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size
Middle (Expected 10 chars): =0.2,  shu
Suffix (At least 5 tokens): ffle=False)
---
Prefix: from sklearn.model_selection impor
Middle (Expected 10 chars): t train_te
Suffix (At least 5 tokens): st_split
---
Prefix: from sklearn.metrics import re
Middle (Expected 10 chars): call_score
Suffix (At least 5 tokens): 
---
Prefix: from sklearn.metrics import acc
Middle (Expected 10 chars): uracy_scor
Suffix (At least 5 tokens): e
---


In [48]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "bigcode/tiny_starcoder_py"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_code_completion(model, tokenizer, prefix, max_new_tokens=10):
    # Tokenize the prefix and check if it has content
    inputs = tokenizer(prefix, return_tensors="pt")
    if inputs["input_ids"].size(1) == 0:  # Check if tokenization resulted in empty input
        return ""  # Return an empty string if the prefix has no tokens

    # Generate exactly 10 tokens
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_new_tokens,  # Set to 10 tokens explicitly
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
completions = []
for prefix, middle, suffix in dataset_aci:
    generated = generate_code_completion(model, tokenizer, prefix)
    completions.append((prefix, middle, generated, suffix))

# Sample output to verify
for prefix, middle, generated, suffix in completions[:5]:  # Show first 5 completions
    print("Prefix:", prefix)
    print("Expected Middle:", middle)
    print("Generated Middle (10 tokens):", generated)
    print("Suffix:", suffix)
    print("---")


Prefix: Created on Wed Oct 20
Expected Middle: 22:57:12 2
Generated Middle (10 tokens): Created on Wed Oct 20 14:45:55 
Suffix: 021
---
Prefix: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size
Expected Middle: =0.2,  shu
Generated Middle (10 tokens): X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4
Suffix: ffle=False)
---
Prefix: from sklearn.model_selection impor
Expected Middle: t train_te
Generated Middle (10 tokens): from sklearn.model_selection importr
from sklearn.metrics import accuracy_score
Suffix: st_split
---
Prefix: from sklearn.metrics import re
Expected Middle: call_score
Generated Middle (10 tokens): from sklearn.metrics import regr_curve
from sklearn.metrics import roc
Suffix: 
---
Prefix: from sklearn.metrics import acc
Expected Middle: uracy_scor
Generated Middle (10 tokens): from sklearn.metrics import acc_score
from sklearn.metrics import confusion_
Suffix: e
---


In [49]:
labeled_completions = []
for prefix, middle, generated, suffix in completions:
    print(f"Prefix:\n{prefix}")
    print(f"Expected (Middle):\n{middle}")
    print(f"Generated:\n{generated}")
    label = input("Label (correct/partial/incorrect): ")
    labeled_completions.append((prefix, middle, generated, suffix, label))

Prefix:
Created on Wed Oct 20
Expected (Middle):
22:57:12 2
Generated:
Created on Wed Oct 20 14:45:55 
Label (correct/partial/incorrect): correct
Prefix:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size
Expected (Middle):
=0.2,  shu
Generated:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4
Label (correct/partial/incorrect): partial
Prefix:
from sklearn.model_selection impor
Expected (Middle):
t train_te
Generated:
from sklearn.model_selection importr
from sklearn.metrics import accuracy_score
Label (correct/partial/incorrect): partial
Prefix:
from sklearn.metrics import re
Expected (Middle):
call_score
Generated:
from sklearn.metrics import regr_curve
from sklearn.metrics import roc
Label (correct/partial/incorrect): incorrect
Prefix:
from sklearn.metrics import acc
Expected (Middle):
uracy_scor
Generated:
from sklearn.metrics import acc_score
from sklearn.metrics import confusion_
Label (correct/partial/incorrect): partial
Pr

In [50]:
!pip install rouge-score

In [52]:
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.chrf_score import sentence_chrf
from rouge_score import rouge_scorer
import Levenshtein
import numpy as np
from scipy.stats import pearsonr

# Compute Metrics
exact_matches = [1 if gen == actual else 0 for _, actual, gen, _ in completions]
bleu_scores = [sentence_bleu([actual], gen, smoothing_function=SmoothingFunction().method1) for _, actual, gen, _ in completions]
chrf_scores = [sentence_chrf(actual, gen) for _, actual, gen, _ in completions]
levenshtein_distances = [Levenshtein.distance(actual, gen) for _, actual, gen, _ in completions]

# ROUGE-L score calculation
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(actual, gen)['rougeL'].fmeasure for _, actual, gen, _ in completions]

# Jaccard Similarity calculation
def jaccard_similarity(actual, gen):
    actual_set = set(actual.split())
    gen_set = set(gen.split())
    return len(actual_set & gen_set) / len(actual_set | gen_set)

jaccard_scores = [jaccard_similarity(actual, gen) for _, actual, gen, _ in completions]

# Print Average Scores
print("Exact Match:", sum(exact_matches) / len(exact_matches))
print("Average BLEU Score:", sum(bleu_scores) / len(bleu_scores))
print("Average chrF Score:", sum(chrf_scores) / len(chrf_scores))
print("Average Levenshtein Distance:", sum(levenshtein_distances) / len(levenshtein_distances))
print("Average ROUGE-L Score:", sum(rouge_l_scores) / len(rouge_l_scores))
print("Average Jaccard Similarity:", sum(jaccard_scores) / len(jaccard_scores))


Exact Match: 0.0
Average BLEU Score: 0.048365785563180816
Average chrF Score: 0.2129106012713658
Average Levenshtein Distance: 49.8
Average ROUGE-L Score: 0.09697361575622444
Average Jaccard Similarity: 0.005


In [51]:
import pandas as pd
from scipy.stats import pearsonr

# Convert manual labels to numerical scores
manual_scores = [2 if label == 'correct' else 1 if label == 'partial' else 0 for _, _, _, _, label in labeled_completions]

# Define metrics and calculate correlations with manual scores
metrics = {
    "Exact Match": exact_matches,
    "BLEU": bleu_scores,
    "chrF": chrf_scores,
    "Levenshtein Distance": [-x for x in levenshtein_distances],  # Invert for positive correlation
    "ROUGE-L": rouge_l_scores,
    "Jaccard Similarity": jaccard_scores,
}

# Calculate and store correlations in a DataFrame
correlation_data = {name: pearsonr(scores, manual_scores)[0] for name, scores in metrics.items()}
correlation_df = pd.DataFrame(correlation_data.items(), columns=['Metric', 'Correlation with Manual Judgment']).sort_values(by="Correlation with Manual Judgment", ascending=False)

# Display the correlations in a readable format
print(correlation_df)


                 Metric  Correlation with Manual Judgment
3  Levenshtein Distance                          0.215319
5    Jaccard Similarity                          0.183344
4               ROUGE-L                          0.014175
2                  chrF                         -0.227366
1                  BLEU                         -0.318060
0           Exact Match                               NaN


<ipython-input-51-c4b30fc875be>:18: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation_data = {name: pearsonr(scores, manual_scores)[0] for name, scores in metrics.items()}


#Raport

Metric Scores:

1.   Exact Match (0.0): Indicates that the model never produced completions exactly matching the expected text. This could mean the model’s outputs are often approximations or variations rather than precise completions.
2.   Average BLEU (0.048) and chrF (0.213): These relatively low scores suggest limited overlap between n-grams (BLEU) and characters (chrF) in the generated and expected completions, reflecting the model's struggle to closely match the target code segments.
3.   Levenshtein Distance (49.8): A high average Levenshtein distance indicates significant differences between the generated and expected texts, requiring many edits to match them.
4.   ROUGE-L (0.097): A low score here points to limited structural overlap between generated and expected completions.
5.   Jaccard Similarity (0.005): Near-zero Jaccard similarity indicates minimal token overlap, showing that generated and expected completions share very few words or phrases.


Correlation with Manual Judgment:

1.   Levenshtein Distance (0.215) and Jaccard Similarity (0.183): These metrics show the highest positive correlations with manual judgment. This suggests that token-level similarity and edit distance better align with human perception of “correctness” or “partial correctness.”
2.   ROUGE-L (0.014): This near-zero correlation indicates that structural overlap does not meaningfully reflect human judgment in this context.
3.   chrF (-0.227) and BLEU (-0.318): Negative correlations suggest that n-gram (BLEU) and character overlap (chrF) are poorly aligned with manual evaluations. This could mean that these metrics don’t capture the essential aspects of code quality that human reviewers prioritize.

#Conclusion

Unfortunetly my first attempts wasn't saved (I had to restart collab after using all CPU and GPU resources and switch to Collab Pro and for some reason my previous version wasn't saved). In this first attempt chrF and BLEU showed quite high score correlation with manula jugment (around 0.6-0.75) chrF and BLEU. In this first example Jaccard Similarity had also respectable score, not as high as BLEU and chrF, but also around(0.55). Not sure why repoduction resulted in that bad scores, need more time for investigation. The deadline is aproaching so I can't reproduce this now, but I do think that using longer prefix and suffix(mulilane) and switching to larger model would benift results (accuracy of generated tokens compared to the middle). I supose adding more variaty to generated test and adding more information would result in better score. I think that generating fixed length is a bad idea. Also filtiring out n noise like comments, TODO, docstrings would also help.